# VGG16

In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime

### Definiciones

In [2]:
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 13  
aint_size  = 3
wfrac_size = 15
wint_size  = 0
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
#wbuffer_size = *word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
#abuffer_size = *word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

   ### Generación de una muestra de fallos

In [3]:
# Decidir si cargar errores de un archivo locs y error_mask o generarlos aleatoriamente
Cargar_errores = True

#print(load_file('Data/Fault Characterization/wgt/Accs_w_707_55'))

if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/locs_0_54')
    error_mask = load_obj('Data/Fault Characterization/error_mask_0_54')
else:
    numero_bits_con_fallo = 15000
    bits_con_fallo = np.random.randint(1,2,numero_bits_con_fallo)
    #crear una mascara (m) del buffer de pesos donde x: bit sin fallo, 0: bit con fallo en 0, 1: bit con fallo en 1.
    #si quieres introducir fallos en activaciones en lugar de los pesos, simplemente cambia wbuffer_size por abuffer_size.
    mbuffer = np.array(['x']*(abuffer_size-numero_bits_con_fallo))
    mbuffer = np.concatenate([mbuffer,bits_con_fallo])
    #distribuir los errores aleatoriamente en la mascara del buffer
    np.random.shuffle(mbuffer)
    #organizar la mascara del buffer por direcciones
    mbuffer = np.reshape(mbuffer,(-1,word_size))
    mbuffer = ["".join(i) for i in mbuffer]
    #filtrar dejando solo las direcciones con error
    locs  = [x for x,y in enumerate(mbuffer) if y.count('x') < 16]
    error_mask = [y for x,y in enumerate(mbuffer) if y.count('x') < 16] 
print('mostrando las 5 primeras direcciones con fallos')
print('direcciones:',locs[0:5])
print('mascara de fallos:',error_mask[0:5])
print(len(error_mask))
print(len(locs))


mostrando las 5 primeras direcciones con fallos
direcciones: [4141, 4181, 4199, 4241, 4249]
mascara de fallos: ['x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x0xxxxxxx0xxxxxx', 'x1xxxxxxx0xxxxxx']
4738
4738


## Cargar el dataset

In [4]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(224,224), 8, trainBatchSize, testBatchSize)

### Crear la red sin fallos

In [5]:
activation_aging = [False]*21


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net1 = GetNeuralNetworkModel('VGG16', (224,224,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net1.load_weights(wgt_dir).expect_partial()
Net1_procces = Net1.get_weights()
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
WeightQuantization(model = Net1, frac_bits = wfrac_size, int_bits = wint_size)
Net1.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
Net1_procces = Net1.get_weights()
loss_sf,acc_sf =Net1.evaluate(test_dataset)

estoy en GetNeuralNetworkModel
aging_active en GetNeuralNetworkModel [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
shape (224, 224, 3)
shape=otras
batch_size 1


C:\Users\usuario\Desktop\CNN_Gating\Nets.py:172: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_error.colums = ['fail_activs','amount_error']


index_list fuera del ciclo 0
errores totales 0
errores capa [0]
index_list_capa [0]
faults_count 0
shape (224, 224, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores totales 0
errores capa [0, 0]
index_list_capa [0, 0]
faults_count 0
shape (224, 224, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores totales 0
errores capa [0, 0, 0]
index_list_capa [0, 0, 0]
faults_count 0
shape (112, 112, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores totales 0
errores capa [0, 0, 0, 0]
index_list_capa [0, 0, 0, 0]
faults_count 0
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores totales 0
errores capa [0, 0, 0, 0, 0]
index_list_capa [0, 0, 0, 0, 0]
faults_count 0
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores totales 0
errores capa [0, 0, 0, 0, 0, 0]
index_list_capa [0, 0, 0, 0, 0, 0]
faults_count 0
shape (56, 56, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 0
errores to

## Función para guardar en dataframe to excel

In [6]:
def same_elements(outputs1,outputs2,ciclo):
    
    list_size_output=[]
    list_output_true=[]
    list_amount_dif=[]
    list_ratio=[]
    list_ratio_zero=[]
    list_total_zero=[]
    numero=[]
    capa=[]
    


    
    for index in range(0,len(outputs2)):
        
        #print('Capa',index,Net2.layers[index].__class__.__name__)
        a=outputs1[index]== outputs2[index]
        size_output=a.size
        output_true=np.sum(a)
        numero.append(index)
        capa.append(Net2.layers[index].__class__.__name__)
        list_output_true.append(output_true)
        list_size_output.append(size_output)
        amount_dif=size_output-output_true
        list_amount_dif.append(amount_dif)
        ratio=((amount_dif*100)/size_output)
        list_ratio.append(ratio)
        non_zero = (np.count_nonzero(outputs2[index]))
        total_zero= size_output-non_zero
        ratio_zero=((total_zero*100)/size_output)
        list_ratio_zero.append(ratio_zero)
        list_total_zero.append(total_zero)

        df_numero=pd.DataFrame(numero)
        df_capa=pd.DataFrame(capa)
        df_list_size_output=pd.DataFrame(list_size_output)
        df_list_output_diff=pd.DataFrame(list_amount_dif)
        df_list_ratio=pd.DataFrame(list_ratio)
        df_list_elem_zero=pd.DataFrame(list_total_zero)
        df_list_ratio_zero=pd.DataFrame(list_ratio_zero)
        
        buf_same_elemen = pd.concat([df_numero,df_capa,df_list_size_output,df_list_output_diff, df_list_ratio, df_list_elem_zero, df_list_ratio_zero], axis=1, join='outer')
        buf_same_elemen.columns = ['num','capa','Total_elements_layer', 'diff_elements', 'Ratio', 'amount_zero', 'Ratio']
        buf_same_elemen.to_excel(writer, sheet_name='ratio_'+ str(ciclo), index=False)
  

### Crear la red con fallos en activaciones en ciclo de vectores true and false

In [7]:
activation_aging = np.array([False]*21)
acc_list=[]
list_ciclo=[]

with pd.ExcelWriter('VGG16/ratio_element_diff.xlsx') as writer:
    


    for i, valor in enumerate(activation_aging):
        ciclo=i
        activation_aging[i]=True 
        activation_aging[i-1]=False    
        print (activation_aging)
    #activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
    #activation_aging= False
        Net2 = GetNeuralNetworkModel('VGG16', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
        Net2.load_weights(wgt_dir).expect_partial()
        loss = tf.keras.losses.CategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
        loss,acc  = Net2.evaluate(test_dataset)
        acc_list.append(acc)
        #list_ciclo.append(i)
        
        X = [x for x,y in test_dataset]
        #salidas del modelo sin fallas para la primer imagen del dataset de prueba
        outputs1= get_all_outputs(Net1,X[0])
        #salidas del modelo con fallas para la primer imagen del dataset de prueba
        outputs2 = get_all_outputs(Net2,X[0])
        #print(outputs1)
        #print(outputs2)
        #print('list_ciclo',list_ciclo)
        #same_elements(outputs1,outputs2,list_ciclo,acc_list)
        same_elements(outputs1,outputs2,ciclo)
writer.close        

        
    
        
print('Ejecución  completada: ', datetime.now().strftime("%H:%M:%S"))   
        
        

[ True False False False False False False False False False False False
 False False False False False False False False False]
estoy en GetNeuralNetworkModel
aging_active en GetNeuralNetworkModel [ True False False False False False False False False False False False
 False False False False False False False False False]
shape (224, 224, 3)
shape=otras
batch_size 1
index_list fuera del ciclo 528
errores totales 1270
errores capa [1270]
index_list_capa [528]
faults_count 528
if active tf.Tensor(True, shape=(), dtype=bool)
shape (224, 224, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360]
index_list_capa [528, 4738]
faults_count 4738
shape (224, 224, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360]
index_list_capa [528, 4738, 4738]
faults_count 4738
shape (112, 112, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores cap

C:\Users\usuario\Desktop\CNN_Gating\Nets.py:172: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_error.colums = ['fail_activs','amount_error']


index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518]
index_list_capa [528, 4738, 4738, 3827]
faults_count 3827
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738]
faults_count 4738
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738]
faults_count 4738
shape (56, 56, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 1634
errores totales 3870
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738, 1634]
faults_count 1634
shape (56, 56, 256)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870, 8518]
index_list_cap

C:\Users\usuario\Desktop\CNN_Gating\Nets.py:172: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_error.colums = ['fail_activs','amount_error']


index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518]
index_list_capa [528, 4738, 4738, 3827]
faults_count 3827
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738]
faults_count 4738
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738]
faults_count 4738
shape (56, 56, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 1634
errores totales 3870
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738, 1634]
faults_count 1634
shape (56, 56, 256)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870, 8518]
index_list_cap

...

index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518]
index_list_capa [528, 4738, 4738, 3827]
faults_count 3827
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738]
faults_count 4738
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738]
faults_count 4738
shape (56, 56, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 1634
errores totales 3870
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738, 1634]
faults_count 1634
shape (56, 56, 256)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870, 8518]
index_list_cap

C:\Users\usuario\Desktop\CNN_Gating\Nets.py:172: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_error.colums = ['fail_activs','amount_error']


index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360]
index_list_capa [528, 4738, 4738]
faults_count 4738
shape (112, 112, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518]
index_list_capa [528, 4738, 4738, 3827]
faults_count 3827
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738]
faults_count 4738
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738]
faults_count 4738
shape (56, 56, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 1634
errores totales 3870
errores capa [1270, 10360, 10360, 8518, 10360, 10360, 3870]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738, 1634]
faults_cou

C:\Users\usuario\Desktop\CNN_Gating\Nets.py:172: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_error.colums = ['fail_activs','amount_error']


index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360]
index_list_capa [528, 4738]
faults_count 4738
shape (224, 224, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360]
index_list_capa [528, 4738, 4738]
faults_count 4738
shape (112, 112, 64)
shape=otras
batch_size 1
index_list fuera del ciclo 3827
errores totales 8518
errores capa [1270, 10360, 10360, 8518]
index_list_capa [528, 4738, 4738, 3827]
faults_count 3827
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738]
faults_count 4738
shape (112, 112, 128)
shape=otras
batch_size 1
index_list fuera del ciclo 4738
errores totales 10360
errores capa [1270, 10360, 10360, 8518, 10360, 10360]
index_list_capa [528, 4738, 4738, 3827, 4738, 4738]
faults_count 4738
shape (56, 56, 128)
shape=otras
batch_size 1
index_li

In [8]:
acc_list


[0.54666668176651,
 0.47333332896232605,
 0.23733332753181458,
 0.19733333587646484,
 0.23333333432674408,
 0.23733332753181458,
 0.2919999957084656,
 0.2133333384990692,
 0.1679999977350235,
 0.17866666615009308,
 0.40533334016799927,
 0.31066668033599854,
 0.30266666412353516,
 0.46266666054725647,
 0.5600000023841858,
 0.5360000133514404,
 0.42933332920074463,
 0.5,
 0.5559999942779541,
 0.5773333311080933,
 0.5773333311080933]

## Crear la red con fallos en activaciones todo en True

In [ ]:
activation_aging = [True]*21

#activation_aging = [False,False,False,False,True,False,False,False,False,False,False]
#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('VGG16', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=activation_aging, word_size=word_size, frac_size=afrac_size, 
                             batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
#WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss_sf,acc_sf =Net2.evaluate(test_dataset)